# Setup

In [1]:
import pandas as pd
from pathlib import Path

In [2]:
#input args
log_name = 'logs/LOG_011.csv'
ticks_per_second_mb = 1000
out_dir = './out'

#derived args
split_logs_dir = f'{out_dir}/split_logs/'

In [3]:
#create output folders
Path(f'{out_dir}/split_logs').mkdir(parents=True, exist_ok=True)
Path(f'{out_dir}/plots').mkdir(parents=True, exist_ok=True)

# Full Log

In [4]:
full_log = pd.read_csv(log_name, delimiter = ';')
full_log.index.rename('Log Entry', inplace = True)
full_log.head()

,Timestamp,Log Entry Type,Data
Log Entry,,,
0,1001,4,Controlling something...
1,1001,3,20202020
2,11,1,"3,1,0,0,0"
3,11,1,"3,2,0,0,0,0,0,0,0"
4,21,1,"3,1,-3232,28820,13603"


In [5]:
#convert timestamp ticks to seconds
full_log['Time_s'] = full_log['Timestamp']/ticks_per_second_mb

In [6]:
#change entry type ids to enums
full_log['Log Entry Type'].replace({1:'SENSOR', 2:'STATE', 3:'ESTIMATOR_VAR', 4:'MSG'}, inplace = True)

# Sensor Log

In [7]:
#extract sensor entries
sensor_log = full_log[full_log['Log Entry Type'] == 'SENSOR'].copy()
#extract data for sensor entries
sensor_log[['Sensor Board ID', 'Sensor Type', 'Data']] = sensor_log['Data'].str.split(',', expand = True, n = 2)

#change sensor type ids to enums
sensor_log['Sensor Type'] = sensor_log['Sensor Type'].astype(int)
sensor_log['Sensor Type'].replace({1:'BARO', 2:'IMU', 3:'GPS'}, inplace = True)

In [8]:
#extract individual sensor types
baro_log = sensor_log[sensor_log['Sensor Type'] == 'BARO'].copy()
imu_log = sensor_log[sensor_log['Sensor Type'] == 'IMU'].copy()
gps_log = sensor_log[sensor_log['Sensor Type'] == 'GPS'].copy()

## Baro Log

In [9]:
#extract data for baro sensors
baro_log[['Pressure', 'Temperature', 'Sensor Board Timestamp']] = baro_log['Data'].str.split(',', expand = True)
baro_log.drop(columns = ['Data'], inplace = True)
#reorder columns
baro_log = baro_log[['Timestamp', 'Time_s', 'Log Entry Type', 'Sensor Board ID', 'Sensor Board Timestamp', 'Sensor Type', 'Pressure', 'Temperature']]

#convert temperature to Celsius
fmt_tmp = lambda x: x / 1000
baro_log['Temperature'] = baro_log['Temperature'].apply(pd.to_numeric).apply(fmt_tmp)
baro_log.head()

,Timestamp,Time_s,Log Entry Type,Sensor Board ID,Sensor Board Timestamp,Sensor Type,Pressure,Temperature
Log Entry,,,,,,,,
2,11,0.011,SENSOR,3,0,BARO,0,0.00
4,21,0.021,SENSOR,3,13603,BARO,-3232,28.82
6,31,0.031,SENSOR,3,13703,BARO,-3232,28.82
8,41,0.041,SENSOR,3,13703,BARO,-3232,28.82
10,51,0.051,SENSOR,3,13703,BARO,-3232,28.82


In [10]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
baro_log.to_csv(f'{split_logs_dir}/baro.csv')

## IMU Log

In [11]:
#generate Acc_x, Acc_y, Acc_z, Gyro_x, Gyro_y, Gyro_z
dim3 = lambda x : [x + '_' + s for s in ['x', 'y', 'z']]
imu_cols = dim3('Acc') + dim3('Gyro')

#extract data for imu sensors
imu_log[imu_cols + ['Sensor Board Timestamp']] = imu_log['Data'].str.split(',', expand = True)
imu_log.drop(columns = ['Data'], inplace = True)
#reorder columns
imu_log = imu_log[['Timestamp', 'Time_s', 'Log Entry Type', 'Sensor Board ID', 'Sensor Board Timestamp', 'Sensor Type'] + imu_cols]

#scale acceleration and rotation to float

#convert to gs
fmt_acc = lambda x: x / 1024
#convert to degrees per second
fmt_gyro = lambda x: x / 65.5

imu_log[dim3('Acc')] = imu_log[dim3('Acc')].apply(pd.to_numeric).apply(fmt_acc)
imu_log[dim3('Gyro')] = imu_log[dim3('Gyro')].apply(pd.to_numeric).apply(fmt_gyro)

imu_log.head()

,Timestamp,Time_s,Log Entry Type,Sensor Board ID,Sensor Board Timestamp,Sensor Type,Acc_x,Acc_y,Acc_z,Gyro_x,Gyro_y,Gyro_z
Log Entry,,,,,,,,,,,,
3,11,0.011,SENSOR,3,0,IMU,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,21,0.021,SENSOR,3,13599,IMU,-0.081055,0.039062,0.944336,0.076336,-1.755725,38.351145
7,31,0.031,SENSOR,3,13599,IMU,-0.081055,0.039062,0.944336,0.076336,-1.755725,38.351145
9,41,0.041,SENSOR,3,13599,IMU,-0.081055,0.039062,0.944336,0.076336,-1.755725,38.351145
11,51,0.051,SENSOR,3,13639,IMU,-0.092773,0.039062,0.955078,-1.709924,-7.114504,39.938931


In [12]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
imu_log.to_csv(f'{split_logs_dir}/imu.csv')

# State Log

In [13]:
#extract state change entries
state_log = full_log[full_log['Log Entry Type'] == 'STATE'].copy()
state_log.head()

,Timestamp,Log Entry Type,Data,Time_s
Log Entry,,,,


In [14]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
state_log.to_csv(f'{split_logs_dir}/state_transitions.csv')

# Estimator Variable Log

In [15]:
#extract estimator var entries
estimator_var_log = full_log[full_log['Log Entry Type'] == 'ESTIMATOR_VAR'].copy()
estimator_var_log.head()

#save to csv; read as pd.read_csv(loc, index_col = 0)
estimator_var_log.to_csv(f'{split_logs_dir}/estimator_vars.csv')

# Message Log

In [16]:
#extract message entries
msg_log = full_log[full_log['Log Entry Type'] == 'MSG'].copy()
msg_log.head()

,Timestamp,Log Entry Type,Data,Time_s
Log Entry,,,,
0,1001,MSG,Controlling something...,1.001
202,2001,MSG,Controlling something...,2.001
404,3001,MSG,Controlling something...,3.001
606,4001,MSG,Controlling something...,4.001
808,5001,MSG,Controlling something...,5.001


In [17]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
msg_log.to_csv(f'{split_logs_dir}/messages.csv')